In [44]:
import pandas as pd
df = pd.read_csv('/content/segmented_trajectories.csv')
df.head()

BaseDateTime       LAT        LON     MMSI PatternDescriptor  \
0  2022-03-31 00:00:17  26.11809 -80.148150  1056261        Stationary   
1  2022-03-31 00:00:27  26.11809 -80.148148  1056261        Stationary   
2  2022-03-31 00:00:37  26.11809 -80.148147  1056261        Stationary   
3  2022-03-31 00:00:47  26.11809 -80.148145  1056261        Stationary   
4  2022-03-31 00:00:57  26.11809 -80.148143  1056261        Stationary   

   Pattern_High Speed  Pattern_Slow Movement  Pattern_Stationary       SOG  \
0                 NaN                    NaN                 1.0  0.100000   
1                 NaN                    NaN                 1.0  0.083607   
2                 NaN                    NaN                 1.0  0.067213   
3                 NaN                    NaN                 1.0  0.050820   
4                 NaN                    NaN                 1.0  0.034426   

   SegmentID  
0          0  
1          0  
2          0  
3          0  
4          0

In [45]:
df[['Pattern_High Speed',	'Pattern_Slow Movement']] = df[['Pattern_High Speed',	'Pattern_Slow Movement']].fillna(0)

In [46]:
df = df.drop(columns=['PatternDescriptor'])

In [47]:
df = df.dropna()

In [48]:
df.isna().sum()

BaseDateTime             0
LAT                      0
LON                      0
MMSI                     0
Pattern_High Speed       0
Pattern_Slow Movement    0
Pattern_Stationary       0
SOG                      0
SegmentID                0
dtype: int64

In [49]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Attention, RepeatVector, TimeDistributed, Concatenate, Dropout, BatchNormalization, Lambda
import matplotlib.pyplot as plt
import os

# Enable device-agnostic compatibility
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    print(f"Available GPUs: {len(physical_devices)}")
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("No GPU found. Running on CPU.")

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Define R² score as a custom metric
def r2_score(y_true, y_pred):
    ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=None)
    y_true_mean = tf.reduce_mean(y_true, axis=None)
    ss_tot = tf.reduce_sum(tf.square(y_true - y_true_mean), axis=None)
    r2 = 1 - (ss_res / (ss_tot + tf.keras.backend.epsilon()))
    return r2

# Add temporal features
def add_temporal_features(data, time_col):
    data[time_col] = pd.to_datetime(data[time_col])
    data['Hour'] = data[time_col].dt.hour
    data['Minute'] = data[time_col].dt.minute
    data['Second'] = data[time_col].dt.second
    data['TimeDelta'] = data[time_col].diff().dt.total_seconds().fillna(0)
    return data

# Prepare sequences with temporal features and SegmentID
def prepare_sequences(data, enc_pattern, enc_segment, window_size, prediction_horizon, group_col='SegmentID', time_col='BaseDateTime'):
    data['Pattern'] = np.argmax(data[['Pattern_High Speed', 'Pattern_Slow Movement', 'Pattern_Stationary']].values, axis=1)

    # Encode pattern and SegmentID
    pattern_encoded = enc_pattern.transform(data[['Pattern']])
    segment_encoded = enc_segment.transform(data[[group_col]])
    data['Pattern_Encoded'] = list(pattern_encoded)
    data['SegmentID_Encoded'] = list(segment_encoded)

    sequences = []
    unique_segments = data[group_col].unique()

    for segment in unique_segments:
        segment_data = data[data[group_col] == segment].reset_index(drop=True)
        for start in range(len(segment_data) - window_size - prediction_horizon + 1):
            x_window = segment_data.iloc[start:start+window_size]
            y_window = segment_data.iloc[start+window_size:start+window_size+prediction_horizon]
            sequences.append({
                'X': {
                    'LAT': x_window['LAT'].tolist(),
                    'LON': x_window['LON'].tolist(),
                    'SOG': x_window['SOG'].tolist(),
                    'Hour': x_window['Hour'].tolist(),
                    'Minute': x_window['Minute'].tolist(),
                    'Second': x_window['Second'].tolist(),
                    'TimeDelta': x_window['TimeDelta'].tolist(),
                    'Patterns': x_window['Pattern_Encoded'].tolist(),
                    'SegmentID': x_window['SegmentID_Encoded'].tolist(),
                },
                'Y': {
                    'LAT': y_window['LAT'].tolist(),
                    'LON': y_window['LON'].tolist(),
                    'SOG': y_window['SOG'].tolist(),
                    'Hour': y_window['Hour'].tolist(),
                    'Minute': y_window['Minute'].tolist(),
                    'Second': y_window['Second'].tolist(),
                    'TimeDelta': y_window['TimeDelta'].tolist(),
                    'Patterns': y_window['Pattern_Encoded'].tolist(),
                    'SegmentID': y_window['SegmentID_Encoded'].tolist(),
                }
            })
    return sequences

# Split data into train/val/test
def split_train_val_test(data, test_size=0.2, val_size=0.2):
    train_data, temp_data = train_test_split(data, test_size=test_size + val_size, random_state=42, shuffle=True)
    val_data, test_data = train_test_split(temp_data, test_size=test_size / (test_size + val_size), random_state=42, shuffle=True)
    return train_data, val_data, test_data

# Data preparation pipeline without scaling
def prepare_data_pipeline_no_scaling(data, window_size, prediction_horizon, test_size=0.2, val_size=0.2):
    # Add Pattern column
    data['Pattern'] = np.argmax(data[['Pattern_High Speed', 'Pattern_Slow Movement', 'Pattern_Stationary']].values, axis=1)

    # Add temporal features
    data = add_temporal_features(data, 'BaseDateTime')

    # Fit encoders for Pattern and SegmentID
    enc_pattern = OneHotEncoder(sparse_output=False)
    enc_segment = OneHotEncoder(sparse_output=False)
    enc_pattern.fit(data[['Pattern']])
    enc_segment.fit(data[['SegmentID']])

    # Split data
    train_data, val_data, test_data = split_train_val_test(data, test_size, val_size)

    # Prepare sequences
    train_sequences = prepare_sequences(train_data, enc_pattern, enc_segment, window_size, prediction_horizon)
    val_sequences = prepare_sequences(val_data, enc_pattern, enc_segment, window_size, prediction_horizon)
    test_sequences = prepare_sequences(test_data, enc_pattern, enc_segment, window_size, prediction_horizon)

    return train_sequences, val_sequences, test_sequences, enc_pattern, enc_segment

# Create model inputs with SegmentID
def create_model_inputs(sequences, observed_seq_len, predicted_seq_len, feature_dim):
    encoder_inputs, decoder_inputs, outputs = [], [], []

    for sequence in sequences:
        X_features = np.array([
            sequence['X']['LAT'], sequence['X']['LON'], sequence['X']['SOG'], sequence['X']['Hour'],
            sequence['X']['Minute'], sequence['X']['Second'], sequence['X']['TimeDelta']
        ]).T
        X_patterns = np.array(sequence['X']['Patterns'])
        X_segments = np.array(sequence['X']['SegmentID'])

        encoder_input = np.concatenate([X_features, X_patterns, X_segments], axis=1)
        decoder_input = np.concatenate([
            np.array([
                sequence['Y']['LAT'], sequence['Y']['LON'], sequence['Y']['SOG'], sequence['Y']['Hour'],
                sequence['Y']['Minute'], sequence['Y']['Second'], sequence['Y']['TimeDelta']
            ]).T,
            np.array(sequence['Y']['Patterns']),
            np.array(sequence['Y']['SegmentID'])
        ], axis=1)

        encoder_inputs.append(encoder_input)
        decoder_inputs.append(decoder_input)
        outputs.append(decoder_input)

    return np.array(encoder_inputs), np.array(decoder_inputs), np.array(outputs)

# Create the model
def create_model(observed_seq_len, predicted_seq_len, feature_dim, hidden_units=128):
    encoder_inputs = Input(shape=(observed_seq_len, feature_dim), name="encoder_inputs")
    encoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, name="encoder_lstm")
    encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)

    attention = Attention(name="attention_layer")
    attention_context = attention([encoder_outputs, encoder_outputs])
    attention_context = Lambda(lambda x: tf.reduce_mean(x, axis=1), name="summarize_attention")(attention_context)
    context_repeated = RepeatVector(predicted_seq_len)(attention_context)

    decoder_inputs = Input(shape=(predicted_seq_len, feature_dim), name="decoder_inputs")
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True, name="decoder_lstm")
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=[encoder_state_h, encoder_state_c])

    merged_context = Concatenate(axis=-1)([decoder_outputs, context_repeated])

    # Add more layers for complexity
    dense_layer_1 = TimeDistributed(Dense(hidden_units * 2, activation="relu"), name="dense_layer_1")(merged_context)
    dense_layer_1 = Dropout(0.4, name="dropout_1")(dense_layer_1)
    dense_layer_1 = BatchNormalization(name="batch_norm_1")(dense_layer_1)

    dense_layer_2 = TimeDistributed(Dense(hidden_units, activation="relu"), name="dense_layer_2")(dense_layer_1)
    dense_layer_2 = Dropout(0.4, name="dropout_2")(dense_layer_2)
    dense_layer_2 = BatchNormalization(name="batch_norm_2")(dense_layer_2)

    dense_layer_3 = TimeDistributed(Dense(hidden_units // 2, activation="relu"), name="dense_layer_3")(dense_layer_2)
    dense_layer_3 = Dropout(0.4, name="dropout_3")(dense_layer_3)
    dense_layer_3 = BatchNormalization(name="batch_norm_3")(dense_layer_3)

    dense_layer_4 = TimeDistributed(Dense(hidden_units // 4, activation="relu"), name="dense_layer_4")(dense_layer_3)
    dense_layer_4 = Dropout(0.4, name="dropout_4")(dense_layer_4)
    dense_layer_4 = BatchNormalization(name="batch_norm_4")(dense_layer_4)

    final_outputs = TimeDistributed(Dense(feature_dim, activation="linear"), name="final_outputs")(dense_layer_4)

    model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=final_outputs, name="complex_attention_lstm_model")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse", metrics=["mae", r2_score])

    return model


# Train function
def train_model(train_encoder_inputs, train_decoder_inputs, train_outputs,
                val_encoder_inputs, val_decoder_inputs, val_outputs,
                model_weights_path, observed_seq_len, predicted_seq_len, feature_dim, epochs=25, batch_size=16):
    if os.path.exists(model_weights_path):
        print("Loading existing model weights...")
        model = create_model(observed_seq_len, predicted_seq_len, feature_dim)
        model.load_weights(model_weights_path)
    else:
        print("No existing weights found. Creating a new model...")
        model = create_model(observed_seq_len, predicted_seq_len, feature_dim)

    # Train the model
    history = model.fit(
        [train_encoder_inputs, train_decoder_inputs], train_outputs,
        validation_data=([val_encoder_inputs, val_decoder_inputs], val_outputs),
        epochs=epochs, batch_size=batch_size, verbose=1
    )

    model.save_weights(model_weights_path)
    print(f"Model weights saved at {model_weights_path}")
    return model

# Assume df is your dataset
window_size = 5
prediction_horizon = 2
train_sequences, val_sequences, test_sequences, enc_pattern, enc_segment = prepare_data_pipeline_no_scaling(df, window_size, prediction_horizon)

observed_seq_len = window_size
predicted_seq_len = prediction_horizon
pattern_dim = enc_pattern.categories_[0].shape[0]
segment_dim = enc_segment.categories_[0].shape[0]
feature_dim = 7 + pattern_dim + segment_dim

train_encoder_inputs, train_decoder_inputs, train_outputs = create_model_inputs(train_sequences, observed_seq_len, predicted_seq_len, feature_dim)
val_encoder_inputs, val_decoder_inputs, val_outputs = create_model_inputs(val_sequences, observed_seq_len, predicted_seq_len, feature_dim)
test_encoder_inputs, test_decoder_inputs, test_outputs = create_model_inputs(test_sequences, observed_seq_len, predicted_seq_len, feature_dim)


model_weights_path = "seq2seq_tf_no_scaling.weights.h5"
trained_model = train_model(train_encoder_inputs, train_decoder_inputs, train_outputs,
                            val_encoder_inputs, val_decoder_inputs, val_outputs,
                            model_weights_path, observed_seq_len, predicted_seq_len, feature_dim)


No GPU found. Running on CPU.
Loading existing model weights...
Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


60/60 ━━━━━━━━━━━━━━━━━━━━ 11s 63ms/step - loss: 0.3597 - mae: 0.0299 - r2_score: 0.9544 - val_loss: 0.0739 - val_mae: 0.0149 - val_r2_score: 0.9882
Epoch 2/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.3362 - mae: 0.0227 - r2_score: 0.9574 - val_loss: 0.0752 - val_mae: 0.0144 - val_r2_score: 0.9880
Epoch 3/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 0.3310 - mae: 0.0224 - r2_score: 0.9588 - val_loss: 0.0736 - val_mae: 0.0143 - val_r2_score: 0.9883
Epoch 4/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - loss: 0.3114 - mae: 0.0220 - r2_score: 0.9612 - val_loss: 0.0681 - val_mae: 0.0141 - val_r2_score: 0.9892
Epoch 5/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 0.3559 - mae: 0.0231 - r2_score: 0.9566 - val_loss: 0.0740 - val_mae: 0.0140 - val_r2_score: 0.9882
Epoch 6/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.3456 - mae: 0.0228 - r2_score: 0.9564 - val_loss: 0.0652 - val_mae: 0.0131 - val_r2_score: 0.9896
Epoch 7/25
60/60 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - los

In [51]:
import matplotlib.pyplot as plt

# Function to test the model on test data
def test_model(model, test_encoder_inputs, test_decoder_inputs, test_outputs, output_file="test_results.png"):
    # Predict outputs
    predictions = model.predict([test_encoder_inputs, test_decoder_inputs], verbose=1)

    # Extract actual and predicted values for LAT and LON
    num_samples = predictions.shape[0]
    num_timesteps = predictions.shape[1]

    actual_latitudes = []
    actual_longitudes = []
    predicted_latitudes = []
    predicted_longitudes = []

    for sample_idx in range(num_samples):
        actual_latitudes.append(test_outputs[sample_idx, :, 0])  # LAT is the first feature
        actual_longitudes.append(test_outputs[sample_idx, :, 1])  # LON is the second feature
        predicted_latitudes.append(predictions[sample_idx, :, 0])
        predicted_longitudes.append(predictions[sample_idx, :, 1])

    # Plot actual vs predicted LAT and LON for a few samples
    # plt.figure(figsize=(12, 6))
    # num_plots = min(5, num_samples)  # Plot for up to 5 samples

    # for i in range(num_plots):
    #     plt.subplot(2, num_plots, i + 1)
    #     plt.plot(range(num_timesteps), actual_latitudes[i], label='Actual LAT')
    #     plt.plot(range(num_timesteps), predicted_latitudes[i], label='Predicted LAT', linestyle='--')
    #     plt.title(f'Sample {i + 1} - LAT')
    #     plt.xlabel('Timesteps')
    #     plt.ylabel('Latitude')
    #     plt.legend()

    #     plt.subplot(2, num_plots, i + 1 + num_plots)
    #     plt.plot(range(num_timesteps), actual_longitudes[i], label='Actual LON')
    #     plt.plot(range(num_timesteps), predicted_longitudes[i], label='Predicted LON', linestyle='--')
    #     plt.title(f'Sample {i + 1} - LON')
    #     plt.xlabel('Timesteps')
    #     plt.ylabel('Longitude')
    #     plt.legend()

    # plt.tight_layout()
    # plt.savefig(output_file)
    # plt.show()
    # print(f"Test results plot saved as {output_file}")

    # Evaluate model performance
    test_loss, test_mae, test_r2 = model.evaluate(
        [test_encoder_inputs, test_decoder_inputs], test_outputs, verbose=1
    )
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test MAE: {test_mae:.4f}")
    print(f"Test R² Score: {test_r2:.4f}")


In [14]:
# Test the trained model
test_model(trained_model, test_encoder_inputs, test_decoder_inputs, test_outputs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1912 - mae: 0.0233 - r2_score: 0.9649
Test Loss: 0.1912
Test MAE: 0.0233
Test R² Score: 0.9649


In [41]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# ------------------ Functions for Input Preparation ------------------

def prepare_single_sequence(observed_data, enc_pattern, enc_segment):
    """
    Prepare a single encoder input sequence (numerical + encoded features).
    """
    numerical_features = observed_data[['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']].values.astype(np.float32)
    pattern_encoded = enc_pattern.transform(observed_data[['Pattern']]).astype(np.float32)
    segment_encoded = enc_segment.transform(observed_data[['SegmentID']]).astype(np.float32)
    combined_features = np.hstack([numerical_features, pattern_encoded, segment_encoded])
    return combined_features

def prepare_decoder_initial_input(last_row, enc_pattern, enc_segment, predicted_seq_len):
    """
    Prepare the initial decoder input with correct dimensions.
    """
    numerical_features = last_row[['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']].values.astype(np.float32)
    numerical_features = np.tile(numerical_features, (predicted_seq_len, 1))
    pattern_encoded = enc_pattern.transform([[last_row['Pattern']]]).astype(np.float32)
    segment_encoded = enc_segment.transform([[last_row['SegmentID']]]).astype(np.float32)

    pattern_repeated = np.tile(pattern_encoded, (predicted_seq_len, 1))
    segment_repeated = np.tile(segment_encoded, (predicted_seq_len, 1))

    combined_features = np.hstack([numerical_features, pattern_repeated, segment_repeated])
    return np.expand_dims(combined_features, axis=0)  # Shape: (1, predicted_seq_len, feature_dim)

# ------------------ Corrected Prediction Function ------------------

def predict_and_format(model, encoder_input, decoder_input_init, observed_data, prediction_horizon, time_delta):
    """
    Perform autoregressive predictions and format them into a DataFrame.
    """
    predictions = []
    decoder_input = decoder_input_init.copy()
    base_datetime = pd.to_datetime(observed_data['BaseDateTime'].iloc[-1])
    mmsi = observed_data['MMSI'].iloc[0]
    segment_id = observed_data['SegmentID'].iloc[0]

    # Initialize with last observed values
    lat = observed_data['LAT'].iloc[-1]
    lon = observed_data['LON'].iloc[-1]
    sog = observed_data['SOG'].iloc[-1]

    # Autoregressive loop for predictions
    for step in range(prediction_horizon):
        # Predict the next step
        prediction = model.predict([encoder_input, decoder_input], verbose=0)
        prediction_step = prediction[:, step, :]  # Extract current step prediction

        # Correct predictions to prevent unrealistic drift
        lat += prediction_step[0, 0] * 0.00001  # Small incremental change
        lon += prediction_step[0, 1] * 0.00001
        sog = max(0, sog + prediction_step[0, 2] * 0.01)  # Ensure SOG stays non-negative

        predictions.append([lat, lon, sog] + prediction_step[0, 3:].tolist())

        # Update decoder input
        next_input = np.expand_dims(prediction_step, axis=1)
        decoder_input = np.concatenate([decoder_input[:, 1:, :], next_input], axis=1)

    predictions = np.array(predictions)

    # Generate future timestamps
    future_datetimes = [base_datetime + pd.Timedelta(seconds=time_delta * (i + 1)) for i in range(prediction_horizon)]

    # Decode patterns
    pattern_pred = np.argmax(predictions[:, 3:6], axis=1)
    pattern_high_speed = (pattern_pred == 0).astype(int)
    pattern_slow_movement = (pattern_pred == 1).astype(int)
    pattern_stationary = (pattern_pred == 2).astype(int)

    # Build output DataFrame
    predicted_df = pd.DataFrame({
        "BaseDateTime": future_datetimes,
        "LAT": predictions[:, 0],
        "LON": predictions[:, 1],
        "SOG": predictions[:, 2],
        "MMSI": [mmsi] * prediction_horizon,
        "Pattern_High Speed": pattern_high_speed,
        "Pattern_Slow Movement": pattern_slow_movement,
        "Pattern_Stationary": pattern_stationary,
        "SegmentID": [segment_id] * prediction_horizon
    })

    return predicted_df

# ------------------ Main Prediction Pipeline ------------------

def preprocess_and_predict(weights_path, data, observed_seq_len, predicted_seq_len, time_delta):
    """
    Preprocess input data, load the model, and perform predictions.
    """
    observed_data = data.iloc[:observed_seq_len]

    # Initialize encoders
    enc_pattern = OneHotEncoder(sparse_output=False).fit(data[['Pattern']])
    enc_segment = OneHotEncoder(sparse_output=False).fit(data[['SegmentID']])

    # Prepare encoder input and decoder initial input
    encoder_input = np.expand_dims(prepare_single_sequence(observed_data, enc_pattern, enc_segment), axis=0)
    decoder_input_init = prepare_decoder_initial_input(observed_data.iloc[-1], enc_pattern, enc_segment, predicted_seq_len)

    # Load model and weights
    model = create_model(observed_seq_len, predicted_seq_len, encoder_input.shape[-1])
    model.load_weights(weights_path)

    # Perform predictions
    predicted_output = predict_and_format(
        model=model,
        encoder_input=encoder_input,
        decoder_input_init=decoder_input_init,
        observed_data=observed_data,
        prediction_horizon=predicted_seq_len,
        time_delta=time_delta
    )

    return observed_data, predicted_output



weights_path = "./seq2seq_tf_no_scaling.weights.h5"
observed_seq_len = 3
predicted_seq_len = 2
time_delta = 10


observed_data, predicted_output = preprocess_and_predict(
    weights_path=weights_path,
    data=df,
    observed_seq_len=observed_seq_len,
    predicted_seq_len=predicted_seq_len,
    time_delta=time_delta
)

# Display results
print("\n===== Observed Data =====")
print(observed_data)

print("\n===== Predicted Output =====")
print(predicted_output)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



===== Observed Data =====
         BaseDateTime       LAT        LON     MMSI  Pattern_High Speed  \
0 2022-03-31 00:00:17  26.11809 -80.148150  1056261                 0.0   
1 2022-03-31 00:00:27  26.11809 -80.148148  1056261                 0.0   
2 2022-03-31 00:00:37  26.11809 -80.148147  1056261                 0.0   

   Pattern_Slow Movement  Pattern_Stationary       SOG  SegmentID  Pattern  \
0                    0.0                 1.0  0.100000          0        2   
1                    0.0                 1.0  0.083607          0        2   
2                    0.0                 1.0  0.067213          0        2   

   Hour  Minute  Second  TimeDelta  
0     0       0      17        0.0  
1     0       0      27       10.0  
2     0       0      37       10.0  

===== Predicted Output =====
         BaseDateTime        LAT        LON       SOG     MMSI  \
0 2022-03-31 00:00:47  26.118408 -80.149052  0.067923  1056261   
1 2022-03-31 00:00:57  26.118730 -80.149965  0.06

In [52]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# ------------------ Functions for Input Preparation ------------------

def prepare_single_sequence(observed_data, enc_pattern, enc_segment):
    """
    Prepare a single encoder input sequence (numerical + encoded features).
    """
    numerical_features = observed_data[['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']].values.astype(np.float32)
    pattern_encoded = enc_pattern.transform(observed_data[['Pattern']]).astype(np.float32)
    segment_encoded = enc_segment.transform(observed_data[['SegmentID']]).astype(np.float32)
    combined_features = np.hstack([numerical_features, pattern_encoded, segment_encoded])
    return combined_features

def prepare_decoder_initial_input(last_row, enc_pattern, enc_segment, predicted_seq_len):
    """
    Prepare the initial decoder input with correct dimensions.
    """
    numerical_features = last_row[['LAT', 'LON', 'SOG', 'Hour', 'Minute', 'Second', 'TimeDelta']].values.astype(np.float32)
    numerical_features = np.tile(numerical_features, (predicted_seq_len, 1))
    pattern_encoded = enc_pattern.transform([[last_row['Pattern']]]).astype(np.float32)
    segment_encoded = enc_segment.transform([[last_row['SegmentID']]]).astype(np.float32)

    pattern_repeated = np.tile(pattern_encoded, (predicted_seq_len, 1))
    segment_repeated = np.tile(segment_encoded, (predicted_seq_len, 1))

    combined_features = np.hstack([numerical_features, pattern_repeated, segment_repeated])
    return np.expand_dims(combined_features, axis=0)  # Shape: (1, predicted_seq_len, feature_dim)

# ------------------ Corrected Prediction Function ------------------

def predict_and_format(model, encoder_input, decoder_input_init, observed_data, prediction_horizon, time_delta):
    """
    Perform autoregressive predictions with smoothing and format into a DataFrame.
    """
    predictions = []
    decoder_input = decoder_input_init.copy()
    base_datetime = pd.to_datetime(observed_data['BaseDateTime'].iloc[-1])
    mmsi = observed_data['MMSI'].iloc[0]
    segment_id = observed_data['SegmentID'].iloc[0]

    # Initialize with last observed values
    lat = observed_data['LAT'].iloc[-1]
    lon = observed_data['LON'].iloc[-1]
    sog = observed_data['SOG'].iloc[-1]

    smoothing_factor = 0.3  # Controls blending between observed and predicted

    # Autoregressive loop for predictions
    for step in range(prediction_horizon):
        # Predict the next step
        prediction = model.predict([encoder_input, decoder_input], verbose=0)
        prediction_step = prediction[:, step, :]  # Extract current step prediction

        # Anchor to the last observed value with small increments
        lat += smoothing_factor * prediction_step[0, 0] * 0.01  # Smaller incremental change
        lon += smoothing_factor * prediction_step[0, 1] * 0.01
        sog = sog + smoothing_factor * (prediction_step[0, 2] - sog)  # Smooth update

        predictions.append([lat, lon, sog] + prediction_step[0, 3:].tolist())

        # Update decoder input
        next_input = np.expand_dims(prediction_step, axis=1)
        decoder_input = np.concatenate([decoder_input[:, 1:, :], next_input], axis=1)

    predictions = np.array(predictions)

    # Generate future timestamps
    future_datetimes = [base_datetime + pd.Timedelta(seconds=time_delta * (i + 1)) for i in range(prediction_horizon)]

    # Decode patterns
    pattern_pred = np.argmax(predictions[:, 3:6], axis=1)
    pattern_high_speed = (pattern_pred == 0).astype(int)
    pattern_slow_movement = (pattern_pred == 1).astype(int)
    pattern_stationary = (pattern_pred == 2).astype(int)

    # Build output DataFrame
    predicted_df = pd.DataFrame({
        "BaseDateTime": future_datetimes,
        "LAT": predictions[:, 0],
        "LON": predictions[:, 1],
        "SOG": predictions[:, 2],
        "MMSI": [mmsi] * prediction_horizon,
        "Pattern_High Speed": pattern_high_speed,
        "Pattern_Slow Movement": pattern_slow_movement,
        "Pattern_Stationary": pattern_stationary,
        "SegmentID": [segment_id] * prediction_horizon
    })

    return predicted_df

# ------------------ Main Prediction Pipeline ------------------

def preprocess_and_predict(weights_path, data, observed_seq_len, predicted_seq_len, time_delta):
    """
    Preprocess input data, load the model, and perform predictions.
    """
    observed_data = data.iloc[:observed_seq_len]

    # Initialize encoders
    enc_pattern = OneHotEncoder(sparse_output=False).fit(data[['Pattern']])
    enc_segment = OneHotEncoder(sparse_output=False).fit(data[['SegmentID']])

    # Prepare encoder input and decoder initial input
    encoder_input = np.expand_dims(prepare_single_sequence(observed_data, enc_pattern, enc_segment), axis=0)
    decoder_input_init = prepare_decoder_initial_input(observed_data.iloc[-1], enc_pattern, enc_segment, predicted_seq_len)

    # Load model and weights
    model = create_model(observed_seq_len, predicted_seq_len, encoder_input.shape[-1])
    model.load_weights(weights_path)

    # Perform predictions
    predicted_output = predict_and_format(
        model=model,
        encoder_input=encoder_input,
        decoder_input_init=decoder_input_init,
        observed_data=observed_data,
        prediction_horizon=predicted_seq_len,
        time_delta=time_delta
    )

    return observed_data, predicted_output


weights_path = "./seq2seq_tf_no_scaling.weights.h5"
observed_seq_len = 3
predicted_seq_len = 10
time_delta = 60

# Perform prediction
observed_data, predicted_output = preprocess_and_predict(
    weights_path=weights_path,
    data=df,
    observed_seq_len=observed_seq_len,
    predicted_seq_len=predicted_seq_len,
    time_delta=time_delta
)

# Display results
print("\n===== Observed Data =====")
print(observed_data)

print("\n===== Predicted Output =====")
print(predicted_output)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 50 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



===== Observed Data =====
         BaseDateTime       LAT        LON     MMSI  Pattern_High Speed  \
0 2022-03-31 00:00:17  26.11809 -80.148150  1056261                 0.0   
1 2022-03-31 00:00:27  26.11809 -80.148148  1056261                 0.0   
2 2022-03-31 00:00:37  26.11809 -80.148147  1056261                 0.0   

   Pattern_Slow Movement  Pattern_Stationary       SOG  SegmentID  Pattern  \
0                    0.0                 1.0  0.100000          0        2   
1                    0.0                 1.0  0.083607          0        2   
2                    0.0                 1.0  0.067213          0        2   

   Hour  Minute  Second  TimeDelta  
0     0       0      17        0.0  
1     0       0      27       10.0  
2     0       0      37       10.0  

===== Predicted Output =====
         BaseDateTime        LAT        LON       SOG     MMSI  \
0 2022-03-31 00:01:37  26.228297 -80.457362  0.064702  1056261   
1 2022-03-31 00:02:37  26.332598 -80.752099  0.06